In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
import scipy 
from scipy.linalg import eigh, cholesky
from scipy.stats import norm
import linearmodels.panel as lmp
from pylab import plot, show, axis, subplot, xlabel, ylabel, grid

%matplotlib inline

# Parte 1 - Experimentos

Deben conceptualizar un experimento con el objetivo de estudiar posibles incentivos o estrategias para incrementar la asistencia a clases en estudiantes universitarios de la UdeC. El outcome del tratamiento es la proporcion promedio de estudiantes que asisten a clases. Todos los elementos del experimento deben ser definidos, respondiendo a las siguientes preguntas: 

1. Asumiendo la existencia de recursos disponibles e implementacion a nivel de estudiante, sugiera un tratamiento que pueda ser testeado a traves de un experimento aleatorizado controlado. Sea especifico en cuanto a los detalles del tratamiento (costos, materiales, duracion, etcetera).

2. Defina los grupos de tratamiento y control para implementar su experimento. Describa en detalle el mecanismo de asignacion aleatorio que permite la comparacion entre grupos.

3. Que metodo considera el mas apropiado para la estimacion del efecto promedio? (pre-test, pre-post test, Salomon 4 group). Justifique su respuesta en base a las ventajas y desventajas de cada metodo. 

4. Ahora suponga que no es posible implementar un experimento a nivel de estudiante, sino a nivel de clase. Como ajustaria los elementos de su experimento para poder ser implementado a nivel de cluster? Sea especifico respecto tanto del tratamiento como del metodo de asignacion aleatorio y potencial comparacion entre grupos de tratamiento y control.

5. Suponga que en vez de un experimento, se planifica que sea un programa implementado a nivel de toda la Universidad. Como ajustaria los elementos descritos anteriormente para poder comparar el efecto de la intervencion.

# Respuestas:

Elementos Básicos:
- Población Objetivo: Estudiantes Udec, primer año, de ramos en que tengan varias secciones.

1. Se propone elegir aleatoriamente un curso por generación en todas las carreras, donde estos cursos sean excluyentes según prerrequisito (para evitar que se repitan estudiantes)  en que se se pase una lista de asistencia para poder registrar el desarrollo del experimento. A partir de la octava semana de clases se propone que la universidad comience a enviar un correo masivo con un boletin de noticias semanal los días domingo, con la particularidad de que a la mitad de los estudiantes de cada asignatura, elegidos aleatoriamente (sorteo por numero de lista), se les agregue al comienzo del boletin un recordatorio de cuanto falta para sus proximos certamenes (si, queremos ver el mundo arder), el cual corresponde al tratamiento, esto se realizará hasta la última semana de clases.

2. (se respondió en la anterior)

3. Descartamos usar diseño salomon y post-test, ya que el diseño del experimento requiere tomar constantemente la asistencia, lo cual puede constituir un pre-test. Por lo tanto se considera más adecuado un diseño pre-post

4. Lo que se cambiaría sería buscar clases en que existieran 2 secciones, de forma que se implementara el tratamiento para una sección completa, mientras que para la otra no. Lo demás se mantiene en la misma forma.

5. Agregando el resto de cursos de toda la universidad, pero ahora eligiendo aleatoriamente no entre curso si no que por estudiante de la carrera, asi por ejemplo la mitad de los estudiantes recibe el tratamiento y la otra mitad no. (aleatorio según numero en la lista de estdudiantes de la carrera)


# Parte 2 - Estimacion de efectos promedio de tratamiento (data simulada)

6. A partir de sus respuestas en Parte 1, genere data para 40 grupos (considere cada grupo como una clase) con 50 estudiantes cada uno (asuma que los estudiantes son asignados aleatoriamente a cada clase). Cada estudiante debe tener data de asistencia en un periodo, generando una variable binaria aleatoria talque la asistencia promedio a traves de todos los grupos es de 80%.

7. Genere un mecanismo de asignacion aleatorio a nivel de estudiante y muestre que en la data generada permite que ambos grupos (tratamiento y control) tienen una asistencia promedio comparable.

8. Genere un tratamiento que imcrementa la participacion en el grupo de tratamiento en 10 puntos porcentuales. Ademas en la data posterior al experimento, asuma que la participacion promedio cayo a 75%. Estime el efecto promedio del tratamiento usando solo post-test.

9. Estime el efecto promedio del tratamiento usando pre-post test con la data generada. Muestre que el efecto es equivalente usando ambos metodos.

10. Estime el efecto ajustando los errores estandar por cluster (la variable grupo representa cada clase). Cual es la diferencia entre ambas estimaciones? Explique porque es esperable (o no) encontrar diferencias entre ambos metodos.


\begin{cases}  
\Phi^{-1}(0.8) = \alpha \\ 
\Phi^{-1}(0.8) = \alpha + \beta \cdot T \\
\Phi^{-1}(0.6) = \alpha + \gamma \cdot p \\
\Phi^{-1}(0.9) = \alpha + \beta \cdot T + \gamma \cdot p + \lambda (T \cdot p)  
\end{cases}

6. asistencia de todos los grupos 0.8
7. a partir de la ecuacion anterior, es posible calcular alpha, beta y gama y lambda, pudiendomo comparar las variables. Se iguala la ecuación del experimento al valor inverso de la distribución normal que sea necesario. Así para ambos pre test se espera un 80% de asistencia, mientras que para el post-test del grupo de tratamiento se espera un aumento de 10 puntos porcentuales, es decir 90% y para el del grupo control 60%, de modo que promedien 75% para el post-test.


In [2]:
from scipy.special import ndtri

# se calcula la inversa de la probabilidad dentro de la distribución normal
pre = ndtri(0.8)
pret = ndtri(0.8)
post = ndtri(0.6) 
postt = ndtri(0.9)

A = np.array([
    [1, 0, 0, 0], 
    [1, 1, 0, 0],
    [1, 0, 1, 0], 
    [1, 1, 1, 1]
    ])

B = np.array([pre, pret, post, postt])
alpha, beta, gama, lamda = np.linalg.inv(A).dot(B)

In [3]:
# experiment parameters
np.random.seed(2022)
nsize = 40 * 50 * 2 # se realizaron los 2 periodos en un solo vector

#pregunta 6
X = norm.rvs(size=(1, nsize))[0]
df = pd.DataFrame(X, columns = ["X"])
df["p"] = 0
df.loc[nsize/2:,"p"] = 1
df["curso"] = [i//50 + 1 for i in range(nsize//2)] * 2

# grupo tratamiento se asigna de forma que el valor sea igual para ambos periodos para cada individuo
df.loc[0:nsize//2-1, "T"] = df.loc[nsize//2:, "T"] = np.random.binomial(1, 0.5, size = nsize//2) 

df["y"] = alpha + beta * df["T"] + gama * df["p"] + lamda * (df["T"] * df["p"]) + df["X"] # ecuacion de pre-post test pero agregando variable de ruido X (normal(0,1))
df["Y"] = df["y"].apply(lambda x: 1 if norm.cdf(x) >= 0.5 else 0) # si estan por sobre la mnedia de la distribución normal entonces asiste, si no no


#df.groupby(["p","T"]).mean()
df.describe()

,X,p,curso,T,y,Y
count,4000.000000,4000.000000,4000.00000,4000.000000,4000.000000,4000.00000
mean,0.017700,0.500000,20.50000,0.492500,0.818380,0.78075
std,0.993094,0.500063,11.54484,0.500006,1.061871,0.41379
min,-3.284991,0.000000,1.00000,0.000000,-2.890857,0.00000
25%,-0.648728,0.000000,10.75000,0.000000,0.102519,1.00000
50%,0.002467,0.500000,20.50000,0.000000,0.810343,1.00000
75%,0.699019,1.000000,30.25000,1.000000,1.542850,1.00000
max,3.469079,1.000000,40.00000,1.000000,4.474132,1.00000


In [ ]:
df.groupby(["curso","p"]).mean()

X     T         y     Y
curso p                                
1     0  0.059145  0.50  0.900766  0.80
      1  0.089935  0.50  0.857384  0.80
2     0 -0.075436  0.48  0.766186  0.82
      1 -0.222910  0.48  0.523975  0.62
3     0 -0.039308  0.46  0.802313  0.72
...           ...   ...       ...   ...
38    1 -0.073382  0.42  0.611811  0.72
39    0  0.114887  0.56  0.956508  0.84
      1  0.177767  0.56  1.006908  0.76
40    0  0.039065  0.50  0.880686  0.80
      1 -0.018749  0.50  0.748700  0.74

[80 rows x 4 columns]

8. se consigue que el promedio sea de 0.75 aproximadamente en la tabla anterior.
para el randomized controlled trial:
  * efecto estandar utilizado de 0.1, tratando de ver que hubo un aumento del  10% en el grupo
  * significancia de 95%
  * posibilidad de obtener falso negativo de 80%

In [ ]:
from statsmodels.stats.power import TTestIndPower

# parameters for power analysis 
effect = 0.1
alpha = 0.05
power = 0.8

# perform power analysis 
analysis = TTestIndPower()
result = analysis.solve_power(effect, power = power, nobs1= None, ratio = 1.0, alpha = alpha)
print('Sample Size: %.3f' % round(result))


Sample Size: 1571.000


In [ ]:
#post-test
y = df.loc[2000:,"y"]
X = df.loc[2000:,"T"]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.221
Model:                            OLS   Adj. R-squared:                  0.221
Method:                 Least Squares   F-statistic:                     567.5
Date:                Sun, 27 Nov 2022   Prob (F-statistic):          1.31e-110
Time:                        16:36:07   Log-Likelihood:                -2823.2
No. Observations:                2000   AIC:                             5650.
Df Residuals:                    1998   BIC:                             5662.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2587      0.031      8.300      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


8. los efectos promedios obtenidos utilizando post test indican que el tratamiento fue significativo, ademas considerando el 
valor promedio antes del test  = 0.2587 
Valor promedio post test = 0.2587 + 1.0582 = 1.3169
Eso ademas debe considerar el error estandar de las variables (0.031 y 0.44)

Indicando que el valor promedio post test aproximadamente igual en prepost test y post-test 

In [ ]:
#pre-post test

y=df['Y']
df['pT']= df['p']*df['T']
X=df[['p','T','pT']]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results2 = model.fit()
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     106.6
Date:                Sun, 27 Nov 2022   Prob (F-statistic):           2.24e-66
Time:                        16:36:07   Log-Likelihood:                -1991.7
No. Observations:                4000   AIC:                             3991.
Df Residuals:                    3996   BIC:                             4017.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8108      0.013     64.854      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


9. utilizando pre post test, el efecto promedio del tratamiento tambien es significativo.

valor promedio pre test = 0.81 - 0.21*1 + 0.3093 * 0
valor promedio post test = 0.81 - 0.21*1 + 0.3093 * 1 = 1.3309

ademas se debe considerar el Error estandar 0.013, 0.018 y 0.025 de p, t, y pT respectivamente ya que son las variables estadistimente significativas (distintas a 0)

In [ ]:
#clustered standard errors
results3 = model.fit(cov_type="cluster", cov_kwds={'groups': df['curso']})
print(results3.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     105.2
Date:                Sun, 27 Nov 2022   Prob (F-statistic):           9.71e-19
Time:                        16:36:07   Log-Likelihood:                -1991.7
No. Observations:                4000   AIC:                             3991.
Df Residuals:                    3996   BIC:                             4017.
Df Model:                           3                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8108      0.013     61.540      0.0

10. utilizando pre post test, el efecto promedio del tratamiento tambien es significativo.

valor promedio pre test = 0.8108 - * -0.2108*1 + 0.3093 * 0 
valor promedio post test = 0.8108 - * -0.21008*1 + 0.3093 * 1 = 1.3309 (+ los errores estandar)

la diferencia entre los metodos es que en el metodo por cluster las observaciones son divididas en pequeños grupos de invididuos vs el que el otro metodo lo hace a nivel individual.
no existen diferencias entre los resultados de ambos metodos. Esto se debe a que el tratamiento no esta correlacionado por el grupo, por lo que al realizar el cluster pre-post test los resultados no deberian cambiar y ser equivalentes.

In [ ]:
try:
    charls = pd.read_csv("../data/charls.csv")
except:
    try:
        charls = pd.read_csv("../../data/charls.csv")
    except:
        charls = pd.read_csv("https://raw.githubusercontent.com/juancaros/LAB-MAA/main/data/charls.csv")

charls["inid"] = charls["inid"].astype("int")
charls.loc[charls.drinkly == ".m", "drinkly"] = None
charls.loc[charls.drinkly == ".d", "drinkly"] = None
charls.loc[charls.drinkly == ".r", "drinkly"] = None
charls.dropna(inplace = True)

aux = charls.value_counts("inid").to_frame().reset_index()
aux.columns = ["inid", "counts"]
inids = aux[aux.counts == 3]["inid"]
charls = charls[charls.inid.isin(inids)]

charls["drinkly"] = charls["drinkly"].astype("int")
charls.dropna(inplace=True)
charls.reset_index(drop=True, inplace=True)

charls.describe()

,age,bnrps,cesd,child,dnrps,drinkly,female,hrsusu,hsize,intmonth,married,nrps,retage,retired,schadj,urban,wave,wealth,inid
count,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.000000,12258.00000,1.225800e+04,12258.000000
mean,58.628080,62.101623,8.482868,2.774351,0.768967,0.347691,0.509055,2.698375,3.634688,7.501795,0.996411,0.546663,1.394926,0.173846,4.329907,0.198238,2.00000,7.542233e+03,12802.149046
std,8.327912,51.712571,6.258322,1.293252,0.421511,0.476257,0.499938,1.702017,1.679174,0.836492,0.059807,0.497838,3.936275,0.378992,3.505935,0.398689,0.81653,4.978348e+04,7877.938460
min,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,-6.190000e+05,1.000000
25%,52.000000,53.335697,4.000000,2.000000,1.000000,0.000000,0.000000,0.000000,2.000000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,2.000000e+02,5081.000000
50%,58.000000,60.000130,7.000000,3.000000,1.000000,0.000000,1.000000,3.555348,3.000000,7.000000,1.000000,1.000000,0.000000,0.000000,4.000000,0.000000,2.00000,1.000000e+03,13467.500000
75%,64.000000,75.692635,12.000000,3.000000,1.000000,1.000000,1.000000,4.025352,5.000000,8.000000,1.000000,1.000000,0.000000,0.000000,8.000000,0.000000,3.00000,9.500000e+03,19855.000000
max,89.000000,300.000000,30.000000,10.000000,1.000000,1.000000,1.000000,5.123964,16.000000,12.000000,1.000000,1.000000,51.000000,1.000000,14.000000,1.000000,3.00000,1.011000e+06,25397.000000


# Parte 3 - Experimentos naturales 

Usando la data **charls.csv**, responda las siguientes preguntas relativas a experimentos naturales.

11. Simule un experimento natural (e.g. intervencion de politica publica) tal que se reduce la proporcion de individuos con 3 hijos o mas que declaran beber alcohol en el tercer periodo a la mitad. Para ello, genere una variable de tratamiento (todos los individuos con mas de 2 hijos son parte de la intervencion), y una nueva variable llamada *sdrinlky*, talque es identica a *drinkly* en los periodos 1 y 2 , pero sustituya los valores aleatoriamente en el periodo 3 para generar el efecto esperado.

In [ ]:
# consumo de alcohol en quienes tienen más de 2 hijos
charls.loc[(charls.wave == 3) & (charls.child > 2) , "drinkly"].value_counts()

0    1483
1     720
Name: drinkly, dtype: int64

In [ ]:
# Simulacion

from scipy.special import ndtri

# se calcula la inversa de la probabilidad dentro de la distribución normal en la misma forma que en las preguntas anteriores
pre = ndtri(charls.loc[(charls.child <= 2), "drinkly"].mean())
pret = ndtri(charls.loc[(charls.child > 2), "drinkly"].mean())
post = ndtri(charls.loc[(charls.child <= 2), "drinkly"].mean()) 
postt = ndtri(charls.loc[(charls.child > 2), "drinkly"].mean() / 2) # se proyecta que el grupo tratramiento disminuya a la mitad su consumo en el post test

A = np.array([
    [1, 0, 0, 0], 
    [1, 1, 0, 0],
    [1, 0, 1, 0], 
    [1, 1, 1, 1]
    ])


B = np.array([pre, pret, post, postt])
alpha, beta, gama, lamda = np.linalg.inv(A).dot(B)

n = len(charls)
charls["T"] = 0
charls.loc[(charls.child > 2), "T"] = 1 # se asigna grupo tratamiento
charls["p"] = 0
charls.loc[(charls.wave == 3), "p"] = 1 # se asigna variable de periodo
charls["X"] = norm.rvs(size=(1, n))[0]

charls["sdrinkly"] = charls["drinkly"] # sdrnkly con valores duplicados

# se cambian los valores de la 3era ola
charls.loc[(charls.wave == 3) , "sdrinkly"] = alpha + beta * charls.loc[(charls.wave == 3) , "T"] + gama * charls.loc[(charls.wave == 3) , "p"] + lamda * (charls.loc[(charls.wave == 3) , "T"] * charls.loc[(charls.wave == 3) , "p"]) + charls.loc[(charls.wave == 3) , "X"] # ecuacion de pre-post test pero agregando variable de ruido

# se toman valores enteros desde la dsitribucion normal, de igual forma al experimento anterior
charls.loc[(charls.wave == 3) , "sdrinkly"] = charls.loc[(charls.wave == 3) , "sdrinkly"].apply(lambda x: 1 if norm.cdf(x) >= 0.5 else 0)

# se muestra como cambia la variable sdrinkly, efectivamente se obtiene un numero cercano a la mitad de drinkly
charls.loc[(charls.wave == 3) & (charls.child > 2) , "sdrinkly"].value_counts()

0.0    1849
1.0     354
Name: sdrinkly, dtype: int64

12. Estime el efecto del tratamiento usando diferencias en diferencias, comparando entre los periodos 2 y 3. 

In [ ]:
# se filtran los periodos
Xa = charls.loc[(charls.wave > 1), ['p','T']]
Xa["pT"] = Xa["p"] * Xa["T"]
ya = charls.loc[(charls.wave > 1), 'sdrinkly']
Xa = sm.add_constant(Xa)

model = sm.OLS(ya, Xa)
results = model.fit(cov_type="HC1")
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               sdrinkly   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     129.5
Date:                Sun, 27 Nov 2022   Prob (F-statistic):           5.97e-82
Time:                        18:16:50   Log-Likelihood:                -5091.7
No. Observations:                8172   AIC:                         1.019e+04
Df Residuals:                    8168   BIC:                         1.022e+04
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3706      0.011     34.096      0.0

Se puede ver como son significativas la constante y el efecto combinado del periodo y el tratamiento, lo cual tiene bastante lógica considerando que se espera que aquellos individuos que no tomaron el tratamiento se les espera un comportamiento similar en ambos periodos. También se encientra significancia para la variable tratamiento, el cual puede deberse a que el grupo de tratamiento fue seleccionado teniendo una demografía especifica, lo cual puede hacer que por pertenecer a este grupo sean distintos.

El efecto de tratamiento y periodo parece es negativo, al igual que el de solo tratamiento, aunque ese ultimo menos significativo y de menor magnitud.

13. Compare el efecto del tratamiento generando grupos pseudo-equivalentes, en particular entre individuos solo con 3 hijos (tratamiento) y 2 hijos (control) 

In [ ]:
charls["group"] = charls["child"].apply(lambda x: 1 if x > 2 else 0)
Xa=charls.loc[(charls.wave > 1), ['p','T','group','female','age','hrsusu','retired']]
Xa["pT"] = Xa["p"] * Xa["T"]
ya=charls.loc[(charls.wave > 1), 'sdrinkly']
Xa = sm.add_constant(Xa)

model = sm.OLS(ya, Xa)
results = model.fit(cov_type="HC1")
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               sdrinkly   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     113.3
Date:                Sun, 27 Nov 2022   Prob (F-statistic):          2.91e-159
Time:                        18:16:50   Log-Likelihood:                -4849.4
No. Observations:                8172   AIC:                             9715.
Df Residuals:                    8164   BIC:                             9771.
Df Model:                           7                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4447      0.044     10.068      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1834: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 7
  'rank is %d' % (J, J_), ValueWarning)


Se agregan más variables a controlar, además se crea una variable grupo.

Las nuevas variablecontroladas corresponden a female, age, hrsusu y retired, de las cuales la unica que resulta ser significativa es female, disminuyendo la prevalencia en el consumo de alcohol en aproximadamente 21 puntos percentuales.

la variable de grupo genera el mismo efecto que la varioable T, dado que en realidad se toma el mismo criterio para su formación

14. Estime el efecto anterior usando la variable *married* como instrumento para determinar el efecto del tratamiento en la pregunta 12. Como se interpreta el efecto en este caso?

In [ ]:
Xa=charls.loc[(charls.wave > 1), ['p','T','female','age','hrsusu','retired']]
Xa["pT"] = Xa["p"] * Xa["T"]
ya=charls.loc[(charls.wave > 1), 'married']
Xa = sm.add_constant(Xa)

model = sm.OLS(ya, Xa)
results = model.fit(cov_type="HC1")
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                married   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     3.018
Date:                Sun, 27 Nov 2022   Prob (F-statistic):            0.00362
Time:                        18:16:50   Log-Likelihood:                 12801.
No. Observations:                8172   AIC:                        -2.559e+04
Df Residuals:                    8164   BIC:                        -2.553e+04
Df Model:                           7                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0065      0.006    179.946      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


En este caso, dado que la variable married no tiene ninguna relación con el tratamiento, las variables T y pT no son significativas para el modelo, ya que no exoplican de ninguna forma el si es que alguien está casado o no. Si se sumaron variables adicinoales para controlar el efecto, donde se tiene como variabkles significativas la edad y el periodo, este ultimo puede deberse a como progresivamente algunos miembros del estudio fueron casandose.

15. Finalmente, asuma que la intervencion se implementa en todos los individuos. Genere una nueva variable de tratamiento un nueva variable llamada *tdrinkly* donde el efecto es una reduccion de 50% en la prevalencia de consumo de alcohol en toda la poblacion en el tercer periodo (identica a *drinkly* en los periodos 1 y 2). Genere una variable *cdrinkly* que es identica a *drinkly* en los periodos 1 y 2 y use la informacion de ambos periodos para predicir el valor esperado de *drinkly* en el tercer periodo, estos seran los valores de *cdrinkly* en el periodo 3 (contrafactual). Finalmente, estime el efecto de la intervencion en toda la poblacion comparando entre *tdrinkly* (datos reales) versus *cdrinkly* contrafactual.  

In [ ]:
charls.loc[(charls.wave == 3) , "drinkly"].value_counts()

0    2681
1    1405
Name: drinkly, dtype: int64

In [ ]:
# Simulacion, se sigue un procedimiento análogo a la pregunta 11

# se calcula la inversa de la probabilidad dentro de la distribución normal
pre = ndtri(charls["drinkly"].mean())
pret = ndtri(charls["drinkly"].mean())
post = ndtri(charls["drinkly"].mean()) 
postt = ndtri(charls["drinkly"].mean() / 2)

A = np.array([
    [1, 0, 0, 0], 
    [1, 1, 0, 0],
    [1, 0, 1, 0], 
    [1, 1, 1, 1]
    ])


B = np.array([pre, pret, post, postt])
alpha, beta, gama, lamda = np.linalg.inv(A).dot(B)

n = len(charls)
charls["T"] = 1
charls["p"] = 0
charls.loc[(charls.wave == 3), "p"] = 1
charls["X"] = norm.rvs(size=(1, n))[0]

charls["tdrinkly"] = charls["drinkly"]

charls.loc[(charls.wave == 3) , "tdrinkly"] = alpha + beta * charls.loc[(charls.wave == 3) , "T"] + gama * charls.loc[(charls.wave == 3) , "p"] + lamda * (charls.loc[(charls.wave == 3) , "T"] * charls.loc[(charls.wave == 3) , "p"]) + charls.loc[(charls.wave == 3) , "X"] # ecuacion de pre-post test pero agregando variable de ruido

charls.loc[(charls.wave == 3) , "tdrinkly"] = charls.loc[(charls.wave == 3) , "tdrinkly"].apply(lambda x: 1 if norm.cdf(x) >= 0.5 else 0)
charls.loc[(charls.wave == 3) , "tdrinkly"].value_counts() # se calcula la variable tdrinkly, como el caso real

0.0    3338
1.0     748
Name: tdrinkly, dtype: int64

In [ ]:
Xf=charls[['female','age','hrsusu','retired']] # se agregan parametros adicionales que son significativos para estimar drinkly
yf=charls['drinkly']
Xf = sm.add_constant(Xf)
model = sm.OLS(yf, Xf)
first = model.fit(cov_type="HC1")

aux = charls.loc[(charls.wave == 3),['female','age','hrsusu','retired']]
aux = sm.add_constant(aux)
charls['cdrinkly'] = charls['drinkly'] 
charls.loc[(charls.wave == 3),'cdrinkly'] = first.predict(aux) # se predice el contrafactual a partie de una regresión abnterior

print(first.summary())

                            OLS Regression Results                            
Dep. Variable:                drinkly   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                     878.9
Date:                Sun, 27 Nov 2022   Prob (F-statistic):               0.00
Time:                        18:22:02   Log-Likelihood:                -6784.3
No. Observations:               12258   AIC:                         1.358e+04
Df Residuals:                   12253   BIC:                         1.362e+04
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6156      0.032     19.049      0.0

In [ ]:
charls.drinkly.mean()

0.3476913036384402

Teniendo tdrinkly como el efecto del tratamiento y cdrinkly como el contrafactual para el cual T = 0 se aplica minimos cuadrados ordinarios para determinar el efecto estimado que tuvo el tratamiento. Al combinar ambas variables estas tienen un efecto significativo en la disminución de cerca de 15.7 puntos percentuales, tomando en cuenta que la propoción de bebedores en la base de datos es de 34.7 % entonces se logra elñ efecto esperado en la población al disminuir en cerca de 50% la prevalencia.

Se encuenytra que la variable de periodo y la variable de tratamiento no tienen significancia por si solas, lo cual es esperable dado que para el periodo cero ambos grupos tienen los mismos variables, además no existen diferencias en el criterio para haber determinado si un individuo pertenece al grupo T= 1 o T =0, ya que realmente se selccionaron a todos los individuos.

Se agregaron variables adicionales que contorlar, de las cuales female, age, hrsusu (horas de trabajo) y retired resultaron ser significativas en estimar si un individuo bebe o no. La primera tiene como efecto disminuir la probabilidad en 31 puntos percentuales, mientras que la edad en 0.1 puntos percentuales, las horas de trabajo tienen un efecto de 1 punto percentual y el estar retirado genera una disminución de 3 puntos percentuales.

In [ ]:
fact = charls.loc[(charls.wave > 1),        ['tdrinkly','p','T', 'child','female','age','hrsusu','retired']]
counterfact = charls.loc[(charls.wave > 1), ['cdrinkly','p','T', 'child','female','age','hrsusu','retired']]
counterfact["T"] = 0
fact.columns =        ['drink','p','T', 'child','female','age','hrsusu','retired']
counterfact.columns = ['drink','p','T', 'child','female','age','hrsusu','retired']

aux= pd.concat([fact, counterfact])
aux["pT"] = aux["p"] * aux["T"]

Xa=aux.drop("drink", axis = 1)
ya=aux["drink"]
Xa = sm.add_constant(Xa)

model = sm.OLS(ya, Xa)
results = model.fit(cov_type="HC1")
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  drink   R-squared:                       0.194
Model:                            OLS   Adj. R-squared:                  0.194
Method:                 Least Squares   F-statistic:                     506.0
Date:                Sun, 27 Nov 2022   Prob (F-statistic):               0.00
Time:                        18:24:11   Log-Likelihood:                -6895.6
No. Observations:               16344   AIC:                         1.381e+04
Df Residuals:                   16335   BIC:                         1.388e+04
Df Model:                           8                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5648      0.026     21.566      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<font size="3">**Tarea 3**</font>

<u> *Instrucciones* </u>

Los resultados de los ejericicios propuestos se deben entregar como un notebook por correo electronico a *juan.caro@uni.lu* el dia 25/11 hasta las 21:00. 

Es importante considerar que el código debe poder ejecutarse en cualquier computadora con la data original del repositorio. Recordar la convencion para el nombre de archivo ademas de incluir en su documento titulos y encabezados por seccion. La unica data real a utilizar en parte de esta tarea es **charls.csv**. El resto de la data de la tarea debe ser generada a partir de las caracteristicas que se especifican. Las variables en **charls.csv** tienen la siguiente descripcion:

- inid: identificador unico
- wave: periodo de la encuesta (1-3)
- cesd: puntaje en la escala de salud mental (0-30)
- child: numero de hijos
- drinkly: bebio alcohol en el ultimo mes (binario)
- hrsusu: horas promedio trabajo semanal
- hsize: tamano del hogar
- intmonth: mes en que fue encuestado/a (1-12)
- married: si esta casado/a (binario)
- retired: si esta pensionado/a (binario)
- schadj: años de escolaridad
- urban: zona urbana (binario)
- wealth: riqueza neta (miles RMB)
- age: edad al entrar a la encuesta (no varia entre periodos)
- bnrps: monto de pension publica (en RMB/mes)
- dnrps: pension implementada en la provincia (binaria)
- retage: fecha esperada de retiro (años desde la fecha de encuenta)
- female: genero del encuestado
- nrps: recibe pension publica

Preguntas:

Parte 1 - Experimentos

Deben conceptualizar un experimento con el objetivo de estudiar posibles incentivos o estrategias para incrementar la asistencia a clases en estudiantes universitarios de la UdeC. El outcome del tratamiento es la proporcion promedio de estudiantes que asisten a clases. Todos los elementos del experimento deben ser definidos, respondiendo a las siguientes preguntas: 

1. Asumiendo la existencia de recursos disponibles e implementacion a nivel de estudiante, sugiera un tratamiento que pueda ser testeado a traves de un experimento aleatorizado controlado. Sea especifico en cuanto a los detalles del tratamiento (costos, materiales, duracion, etcetera).

2. Defina los grupos de tratamiento y control para implementar su experimento. Describa en detalle el mecanismo de asignacion aleatorio que permite la comparacion entre grupos.

3. Que metodo considera el mas apropiado para la estimacion del efecto promedio? (pre-test, pre-post test, Salomon 4 group). Justifique su respuesta en base a las ventajas y desventajas de cada metodo. 

4. Ahora suponga que no es posible implementar un experimento a nivel de estudiante, sino a nivel de clase. Como ajustaria los elementos de su experimento para poder ser implementado a nivel de cluster? Sea especifico respecto tanto del tratamiento como del metodo de asignacion aleatorio y potencial comparacion entre grupos de tratamiento y control.

5. Suponga que en vez de un experimento, se planifica que sea un programa implementado a nivel de toda la Universidad. Como ajustaria los elementos descritos anteriormente para poder comparar el efecto de la intervencion.  

Parte 2 - Estimacion de efectos promedio de tratamiento (data simulada)

6. A partir de sus respuestas en Parte 1, genere data para 40 grupos (considere cada grupo como una clase) con 50 estudiantes cada uno (asuma que los estudiantes son asignados aleatoriamente a cada clase). Cada estudiante debe tener data de asistencia en un periodo, generando una variable binaria aleatoria talque la asistencia promedio a traves de todos los grupos es de 80%.

7. Genere un mecanismo de asignacion aleatorio a nivel de estudiante y muestre que en la data generada permite que ambos grupos (tratamiento y control) tienen una asistencia promedio comparable.

8. Genere un tratamiento que imcrementa la participacion en el grupo de tratamiento en 10 puntos porcentuales. Ademas en la data posterior al experimento, asuma que la participacion promedio cayo a 75%. Estime el efecto promedio del tratamiento usando solo post-test.

9. Estime el efecto promedio del tratamiento usando pre-post test con la data generada. Muestre que el efecto es equivalente usando ambos metodos.

10. Estime el efecto ajustando los errores estandar por cluster (la variable grupo representa cada clase). Cual es la diferencia entre ambas estimaciones? Explique porque es esperable (o no) encontrar diferencias entre ambos metodos.

Parte 3 - Experimentos naturales 

Usando la data **charls.csv**, responda las siguientes preguntas relativas a experimentos naturales.

11. Simule un experimento natural (e.g. intervencion de politica publica) tal que se reduce la proporcion de individuos con 3 hijos o mas que declaran beber alcohol en el tercer periodo a la mitad. Para ello, genere una variable de tratamiento (todos los individuos con mas de 2 hijos son parte de la intervencion), y una nueva variable llamada *sdrinlky*, talque es identica a *drinkly* en los periodos 1 y 2 , pero sustituya los valores aleatoriamente en el periodo 3 para generar el efecto esperado.

12. Estime el efecto del tratamiento usando diferencias en diferencias, comparando entre los periodos 2 y 3. 

13. Compare el efecto del tratamiento generando grupos pseudo-equivalentes, en particular entre individuos solo con 3 hijos (tratamiento) y 2 hijos (control). 

14. Estime el efecto anterior usando la variable *married* como instrumento para determinar el efecto del tratamiento en la pregunta 12. Como se interpreta el efecto en este caso?

15. Finalmente, asuma que la intervencion se implementa en todos los individuos. Genere una nueva variable de tratamiento un nueva variable llamada *tdrinkly* donde el efecto es una reduccion de 50% en la prevalencia de consumo de alcohol en toda la poblacion en el tercer periodo (identica a *drinkly* en los periodos 1 y 2). Genere una variable *cdrinkly* que es identica a *drinkly* en los periodos 1 y 2 y use la informacion de ambos periodos para predicir el valor esperado de *drinkly* en el tercer periodo, estos seran los valores de *cdrinkly* en el periodo 3 (contrafactual). Finalmente, estime el efecto de la intervencion en toda la poblacion comparando entre *tdrinkly* (datos reales) versus *cdrinkly* contrafactual.   

In [ ]:
counterfact

,drink,p,T,child,female,age,hrsusu,retired
1,0.000000,0,0,2,1,48,3.891820,0
2,0.184018,1,0,2,1,50,4.025352,0
4,1.000000,0,0,2,0,50,3.891820,0
5,0.608110,1,0,2,0,52,4.025352,0
7,1.000000,0,0,1,0,49,3.737670,0
...,...,...,...,...,...,...,...,...
12251,0.160872,1,0,3,1,53,3.044522,0
12253,1.000000,0,0,1,0,49,3.465736,0
12254,0.614487,1,0,2,0,48,4.025352,0
12256,0.000000,0,0,1,1,51,2.890372,0


#Ideas

Tratamiento 1: Nota por asistencia entregando syllabus distintos a distintas secciones
Tratamiento 2: Correo Spam eligiendo aleatoriamente a los estudiantes.
Tratamiento 3: Probar distintos horarios.
Tratamiento 4: Descuento de arancel por maxima racha acumulada de asistencia.